<a href="https://colab.research.google.com/github/lrssv/ComputacaoEvolutiva/blob/master/Initial_population.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model for classifying the victim's gender using an evolutionary algorithm to select features.

* ## Initial Population

##Imports

In [ ]:
!pip install category_encoders

In [ ]:
import pandas as pd
import numpy as np
import random
import pickle
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from pandas import read_csv 
from category_encoders import TargetEncoder

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Scripts

In [ ]:
def database(url):
  df_encoded = pd.read_csv(url)
  df_encoded = df_encoded.drop(columns=['Unnamed: 0'])

  return df_encoded

In [ ]:
def population(ind_size,pop_size,models):
  ind = []
  pop = []

  for p in range(pop_size):
    ind = [random.choice([True,False]) for i in range(ind_size)]
    ind[0] = random.choice(range(0,len(models)+1))
    pop.append(ind)
    ind = []

  return pop

## Instances

In [ ]:
df_encoded = database('/content/drive/My Drive/Data Files/df_model_victim_encoded')

In [ ]:
models = [DecisionTreeClassifier(max_depth=10),DecisionTreeClassifier(max_depth=3),DecisionTreeClassifier(max_depth=14),RandomForestClassifier(max_depth=10)]

In [ ]:
initial_population = population(18,100,models)

In [ ]:
#export 
with open("/content/drive/My Drive/Data Files/initial_population.txt", "wb") as fp: 
  pickle.dump(initial_population, fp)